## The ROM bootloader

The ROM bootloader is included during manufacturing of the integrated circuit, and it will be identical for all CC2640R2F chips that belong to the same hardware revision.

The ROM bootloader is the first piece of code that will be executed when the device is powered-on or reset. 
This ROM bootloader will prepare the device such that the main application code can be executed. The ROM bootloader will also enable or disable the JTAG peripheral according to the settings stored in FLASH memory. Additionally, the bootloader hosts a Serial Bootloader Interface that allows us to perform basic actions such as reading/writing memory through a serial interface. 

In this notebook we will provide a basic example that allows to interact with the serial bootloader interface. We will use this example to dump the ROM bootloader, allowing us to analyse the code in Ghidra.

For a full description of the ROM bootloader serial interface we refer to the [CC2538/CC26x0/CC26x2 Serial Bootloader Interface application report](https://www.ti.com/lit/an/swra466d/swra466d.pdf) and the [CC13x0, CC26x0 SimpleLink Wireless MCU Technical Reference Manual](https://www.ti.com/lit/ug/swcu117i/swcu117i.pdf).

This application report includes the following boot flowchart. As you can see we can reach the "Enter ROM bootloader (Serves commands if BL enabled)" box by making sure that the FLASH image valid parameter stored in the Customer Configuration (CCFG) is cleared. The most straightforward way to achieve this is to simply erase microcontroller's flash memory.

You can run this notebook with an unmodified LAUNCHXL-CC2640R2 development board. 

![Boot flowchart](img/boot_flowchart.png)

In [1]:
# Modify the dslite_path variable to point to your installation of Uniflash

import subprocess
from pathlib import Path

home_dir = str(Path.home()) 
dslite_path = home_dir + '/ti/uniflash_7.0.0/dslite.sh'
erase_cmd = dslite_path + ' --mode cc13xx-cc26xx-mass-erase -d XDS110'

In [2]:
# !!! Executing this cell will erase the flash memory (and thus firmware) of your microcontroller !!!
# If you have a copy of the firmware currently on the device you can simply program it back later

process = subprocess.Popen(erase_cmd.split(' '), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output = process.communicate()

if b'Device Unlocked' not in output[0]:
    print('There was an error while trying to erase the microcontroller')
    print(output)
else:
    print(output[0].decode('utf-8'))

Executing the following command:
> /home/lennert/ti/uniflash_7.0.0/deskdb/content/TICloudAgent/linux/ccs_base/DebugServer/bin/DSLite cc13xx-cc26xx-mass-erase -d XDS110

For more details and examples, please refer to the UniFlash Quick Start guide.

Performing Device Unlock via Mass Erase...
Device Unlocked



In [3]:
import serial
import time
from tqdm.notebook import tqdm

class SerialBootloaderInterface:
    
    def __init__(self, port='/dev/ttyACM0'):
        self.ACK = bytes([0x00, 0xcc])
        self.NACK = bytes([0x00, 0x33])
        self.status_val = {
            'COMMAND_RET_SUCCESS': 0x40,
            'COMMAND_RET_UNKNOWN_CMD': 0x41,
            'COMMAND_RET_INVALID_CMD': 0x42,
            'COMMAND_RET_INVALID_ADR': 0x43,
            'COMMAND_RET_FLASH_FAIL': 0x44
        } 
        
        self.ser = serial.Serial(port, 115200)
        self.ser.write(bytes([0x55, 0x55]))
        time.sleep(0.1)
        ret = self.ser.read(self.ser.in_waiting)
        
        if ret != self.ACK:
            raise ValueError('Expected the bootloader to reply with 00CC, instead it replied with: ' + ret.hex())
        else:
            print('Connected to the Serial Bootloader Interface!')
            print('Chip identifier:', self.get_chip_id().hex())
        
        
    def get_chip_id(self):
        self.ser.flushInput()
        self.ser.write(bytes([0x03, 0x28, 0x28]))
        time.sleep(0.1)
        resp = self.ser.read(self.ser.in_waiting)
        self.ser.write(bytes([0xcc]))
        return resp
    
    
    def ping(self):
        self.ser.flushInput()
        self.ser.write(bytes([0x03, 0x20, 0x20]))
        time.sleep(0.1)
        ret = self.ser.read(self.ser.in_waiting)
        assert ret == self.ACK, 'Ping command was not acknowledged by device'
        return ret
    
    
    def get_status(self):
        self.ser.flushInput()
        self.ser.write(bytes([0x03, 0x23, 0x23]))
        time.sleep(0.1)
        resp = self.ser.read(self.ser.in_waiting)
        self.ser.write(bytes([0xcc])) 
        return list(resp)[-1]
    
    
    def read_memory_block(self, addr, no_of_access=253):
        size = 9
        cmd = 0x2A
        access_type = 0x0
        
        payload = bytearray([cmd])
        payload += bytearray(addr.to_bytes(4, byteorder='big'))
        payload += bytearray([access_type, no_of_access])

        chk = 0
        for b in payload:
            chk += b
        chk = chk & 0xff

        packet = bytearray([size, chk]) + payload

        self.ser.flushInput()
        self.ser.write(packet)
        time.sleep(0.1)
        resp = self.ser.read(self.ser.in_waiting)
        self.ser.write([0x00, 0xcc])

        ind = resp.find(0xcc, 0) + 3 
        resp = resp[ind:ind+no_of_access]
        
        status = self.get_status()
        if status != self.status_val['COMMAND_RET_SUCCESS']:
            raise ValueError('ERROR! status:' + 
                             list(self.status_val.keys())[list(self.status_val.values()).index(status)])
        
        assert len(resp) == no_of_access, "Did not receive all bytes..."
        return resp
        
        
    def read_memory_region(self, addr_start, addr_stop):
        blocks = (addr_stop-addr_start)//253
        data = bytearray()

        print('Reading', blocks, 'blocks of data starting from address', hex(addr_start))
        for i in tqdm(range(blocks)):
            block = self.read_memory_block(addr_start, no_of_access=253)
            data += block
            addr_start += 253

        print('reading final block of size', addr_stop-addr_start)
        block = self.read_memory_block(addr_start, addr_stop-addr_start)
        data += block
        return data

In [5]:
# You might have to reset the microcontroller before executing this cell
# You can do this using the reset button on the development board
# Make sure to change the serial port if needed

sbi = SerialBootloaderInterface(port='/dev/ttyACM0')
ret = sbi.ping()

Connected to the Serial Bootloader Interface!
Chip identifier: 00cc0642b0029000


The following memory map is part of the [documentation](https://software-dl.ti.com/lprf/simplelink_cc2640r2_latest/docs/blestack/ble_user_guide/html/cc2640/memory_management.html) provided by Texas Instruments. From this memory map it is clear that the ROM region (in orange) starts at address 0x10000000 and is 0x1CC00 bytes in size.

By executing the next cell we will use the ROM bootloader's serial interface to read out the ROM memory region!

![Boot flowchart](img/cc2640_mem_map.png)

In [6]:
data = sbi.read_memory_region(addr_start=0x10000000, addr_stop=0x1001cc00)

with open('rom_bootloader_cc2640r2.bin', 'wb') as f: # Destination of our ROM dump
    f.write(data)

Reading 465 blocks of data starting from address 0x10000000


  0%|          | 0/465 [00:00<?, ?it/s]

reading final block of size 115


In [7]:
!strings -n 15 rom_bootloader_cc2640r2.bin

(c) Copyright 2005-2015 Texas Instruments Incorporated.  All rights reserved.
Fxdc.runtime.Startup: 'maxPasses' exceeded
Fxdc.runtime.Error.raise: terminating execution


Great! We have now obtained a copy of the ROM bootloader, we can analyse this bootloader in Ghidra to understand in more detail how JTAG is being disabled/enabled. Additionally one could try to identify vulnerabilities in the handling of serial commands and check if there are any undocumented commands!

The README file in the repository contains some basic instructions on how to start analysing the ROM bootloader in Ghidra.

## Dumping the CC2652R1F bootloader

Similarly, the following cells can be used to dump the ROM bootloader of a CC2652R1F. We can reuse the serial bootloader interface code from before, but we do change the address range to be dumped according to the [memory map](https://software-dl.ti.com/lprf/simplelink_cc26x2_latest/docs/ble5stack/ble_user_guide/html/ble-stack-5.x/memory_management.html).

In [8]:
# !!! Executing this cell will erase the flash memory (and thus firmware) of your microcontroller !!!
# If you have a copy of the firmware currently on the device you can simply program it back later

process = subprocess.Popen(erase_cmd.split(' '), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output = process.communicate()

if b'Device Unlocked' not in output[0]:
    print('There was an error while trying to erase the microcontroller')
    print(output)
else:
    print(output[0].decode('utf-8'))

Executing the following command:
> /home/lennert/ti/uniflash_7.0.0/deskdb/content/TICloudAgent/linux/ccs_base/DebugServer/bin/DSLite cc13xx-cc26xx-mass-erase -d XDS110

For more details and examples, please refer to the UniFlash Quick Start guide.

Performing Device Unlock via Mass Erase...
Device Unlocked



In [11]:
# You might have to reset the microcontroller before executing this cell

sbi = SerialBootloaderInterface(port='/dev/ttyACM1')
ret = sbi.ping()

Connected to the Serial Bootloader Interface!
Chip identifier: 00cc06223002f000


In [12]:
data = sbi.read_memory_region(addr_start=0x10000000, addr_stop=0x10000000 + 0x40000)

with open('rom_bootloader_cc2652r1.bin', 'wb') as f: # Destination of our ROM dump
    f.write(data)

Reading 1036 blocks of data starting from address 0x10000000


  0%|          | 0/1036 [00:00<?, ?it/s]

reading final block of size 36


In [13]:
!strings -n 10 rom_bootloader_cc2652r1.bin

(c) Copyright 2005-2015 Texas Instruments Incorporated.  All rights reserved.
*2-'SEL9>S
0BTpG 0BTpG
h^j@GPm|_}
hy qhx`p(F
G0x(r!Fpxhr
 Bogus Exception return value: %08x.
Exception occurred in background thread at PC = 0x%08x.
Exception occurred in ISR thread at PC = 0x%08x.
FCore %d: Exception occurred in ThreadType_%s.
%s name: %s, handle: 0x%x.
%s stack base: 0x%x.
F%s stack size: 0x%x.
FR0 = 0x%08x  R8  = 0x%08x
R1 = 0x%08x  R9  = 0x%08x
R2 = 0x%08x  R10 = 0x%08x
R3 = 0x%08x  R11 = 0x%08x
R4 = 0x%08x  R12 = 0x%08x
R5 = 0x%08x  SP(R13) = 0x%08x
R6 = 0x%08x  LR(R14) = 0x%08x
R7 = 0x%08x  PC(R15) = 0x%08x
PSR = 0x%08x
FICSR = 0x%08x
MMFSR = 0x%02x
BFSR = 0x%02x
UFSR = 0x%04x
HFSR = 0x%08x
DFSR = 0x%08x
MMAR = 0x%08x
BFAR = 0x%08x
AFSR = 0x%08x
IBUSERR: Instruction Access Violation
FPRECISERR: Immediate Bus Fault, exact addr known
IMPRECISERR: Delayed Bus Fault, exact addr unknown
UNSTKERR: Bus Fault caused by Stack Pop
STKERR: Bus Fault caused by Stack Push
FUNALIGNED: Unaligned memo